In [ ]:
import torch
from torch import nn
from torchvision import models
import numpy as np
from scipy import linalg
import os
import glob
import cv2
from PIL import Image
# Load Inception model
inception = models.inception_v3(pretrained=True).eval()
inception.fc = nn.Identity()  # Remove last layer to get features

def get_features(images, model, batch_size=32):
    features = []
    for i in range(0, len(images), batch_size):
        batch = torch.from_numpy(images[i:i + batch_size])
        batch = batch.permute(0, 3, 1, 2)
        with torch.no_grad():
            batch_features = model(batch).cpu().numpy()
        features.append(batch_features)
    return np.concatenate(features)

def calculate_fid(real_features, generated_features):
    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = generated_features.mean(axis=0), np.cov(generated_features, rowvar=False)
    diff = mu1 - mu2
    covmean = linalg.sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid


def images_to_numpy_array(directory_path, resize_shape=None):
    """
    Loads all images in directory_path into a single NumPy array.

    Parameters:
    -----------
    directory_path: str
        Path to the directory containing the images.
    resize_shape: (width, height) tuple or None
        If not None, resize each image to this shape.

    Returns:
    --------
    np.ndarray
        A NumPy array of shape (num_images, height, width, channels).
    """

    # Identify all image files in the directory
    # You can adjust the glob pattern for jpg, png, etc.
    image_files = glob.glob(os.path.join(directory_path, "*.jpg"))
    image_files = glob.glob(os.path.join(directory_path, "*.jpeg"))
    image_files += glob.glob(os.path.join(directory_path, "*.png"))
    image_files += glob.glob(os.path.join(directory_path, "*.webp"))
    images_list = []

    for image_file in image_files:
        # Open each image
        with Image.open(image_file) as img:
            # Optionally resize
            img = img.resize((299,299))
            # Convert to NumPy array (automatically channels=3 for RGB or channels=4 for RGBA, etc.)
            img = img.convert('RGB')
            img_array = np.array(img)

            images_list.append(img_array)

    # Convert the list to a NumPy array
    # Shape would be (N, height, width, channels)
    # If images have different shapes and you haven't resized them, this step can fail
    print(np.array(images_list).shape)
    images_np = np.stack(images_list, axis=0)

    return images_np
real_images = images_to_numpy_array('/content/drive/MyDrive/Colab Notebooks/test images/10_random_images')
generated_images = images_to_numpy_array('/content/drive/MyDrive/Colab Notebooks/test images/lora result imags')

real_features = get_features(real_images, inception)
generated_features = get_features(generated_images, inception)

(0,)


ValueError: need at least one array to stack

In [2]:
import torch
from torch import nn
from torchvision import models
import numpy as np
from scipy import linalg
import os
import glob
import cv2
from PIL import Image
# Load Inception model
inception = models.inception_v3(pretrained=True).eval()
inception.fc = nn.Identity()  # Remove last layer to get features

def get_features(images, model, batch_size=32):
    features = []
    for i in range(0, len(images), batch_size):
        batch = torch.from_numpy(images[i:i + batch_size])
        batch = batch.permute(0, 3, 1, 2)
        with torch.no_grad():
            batch_features = model(batch).cpu().numpy()
        features.append(batch_features)
    return np.concatenate(features)

def calculate_fid(real_features, generated_features):
    mu1, sigma1 = real_features.mean(axis=0), np.cov(real_features, rowvar=False)
    mu2, sigma2 = generated_features.mean(axis=0), np.cov(generated_features, rowvar=False)
    diff = mu1 - mu2
    covmean = linalg.sqrtm(sigma1.dot(sigma2))
    if np.iscomplexobj(covmean):
        covmean = covmean.real
    fid = diff.dot(diff) + np.trace(sigma1 + sigma2 - 2 * covmean)
    return fid


def images_to_numpy_array(directory_path, resize_shape=None):
    """
    Loads all images in directory_path into a single NumPy array.

    Parameters:
    -----------
    directory_path: str
        Path to the directory containing the images.
    resize_shape: (width, height) tuple or None
        If not None, resize each image to this shape.

    Returns:
    --------
    np.ndarray
        A NumPy array of shape (num_images, height, width, channels).
    """

    # Identify all image files in the directory
    # You can adjust the glob pattern for jpg, png, etc.
    image_files = glob.glob(os.path.join(directory_path, "*.jpg"))
    image_files.extend(glob.glob(os.path.join(directory_path, "*.jpeg"))) # Use extend instead of assignment to accumulate files
    image_files.extend(glob.glob(os.path.join(directory_path, "*.png"))) # Use extend instead of assignment to accumulate files
    image_files.extend(glob.glob(os.path.join(directory_path, "*.webp"))) # Use extend instead of assignment to accumulate files
    images_list = []

    for image_file in image_files:
        # Open each image
        with Image.open(image_file) as img:
            # Optionally resize
            img = img.resize((299,299))
            # Convert to NumPy array (automatically channels=3 for RGB or channels=4 for RGBA, etc.)
            img = img.convert('RGB')
            img_array = np.array(img)

            images_list.append(img_array)

    # Convert the list to a NumPy array
    # Shape would be (N, height, width, channels)
    # If images have different shapes and you haven't resized them, this step can fail
    print(np.array(images_list).shape)

    # Check if images_list is empty before stacking
    if not images_list:
        raise ValueError(f"No images found in directory: {directory_path}. Please check the directory path and file extensions.")

    images_np = np.stack(images_list, axis=0)

    return images_np
real_images = images_to_numpy_array('/content/drive/MyDrive/539/ECE539/CelebAMask-HQ_500/data/images')
generated_images = images_to_numpy_array('/content/drive/MyDrive/539/ECE539/inpaint_lora/image results')

real_features = get_features(real_images, inception)
generated_features = get_features(generated_images, inception)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=Inception_V3_Weights.IMAGENET1K_V1`. You can also use `weights=Inception_V3_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/inception_v3_google-0cc3c7bd.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-0cc3c7bd.pth
100%|██████████| 104M/104M [00:01<00:00, 72.2MB/s] 


(492, 299, 299, 3)
(360, 299, 299, 3)


In [3]:
calculate_fid(real_features, generated_features)

np.float64(282.6513325250974)

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive
